In [3]:
from tqdm import tqdm
from lib.utils import load_raw_list
import os
import torch
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from torch.nn.functional import one_hot

normalize = True
balance = True
windowsize = 9
y_train_all = torch.Tensor()
y_test_all = torch.Tensor()
x_train_i = 0
x_test_i = 0
data_dir = f'w{windowsize}_bilstm'
os.makedirs(data_dir)
os.makedirs(f'{data_dir}/train')
os.makedirs(f'{data_dir}/test')

for fileindex in tqdm(range(32)):
    X,y = load_raw_list([fileindex])

    if(normalize):
        # center, stretch
        X = (X - X.mean(axis=1,keepdim=True))/X.std(axis=1,keepdim=True)
        # drop row if any element is inf
        not_inf_idx = torch.where(~X.isinf().any(axis=1))[0]
        X,y = X[not_inf_idx], y[not_inf_idx]

    cat = [X[:-(windowsize-1)]]

    for i in range(1,(windowsize-1)):
        cat.append(X[i:i-(windowsize-1)])
    cat.append(X[(windowsize-1):])

    X = torch.cat(cat,axis=1).float()
    y = y[windowsize//2:-(windowsize//2)]

    X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=.5,shuffle=True,stratify=y,random_state=0)

    if (balance):
        sm = SMOTE(random_state=0)
        X_train_res, y_train_res = sm.fit_resample(X_train, y_train.argmax(axis=1))

    X_train_res = torch.from_numpy(X_train_res)
    y_train_res_oh = one_hot(torch.from_numpy(y_train_res)).float()


    y_train_all = torch.cat([y_train_all,y_train_res_oh])
    y_test_all = torch.cat([y_test_all,y_test])

    for Xi in X_train_res:
        torch.save(Xi.clone(),f'{data_dir}/train/{x_train_i}.pt')
        x_train_i += 1
    for Xi in X_test:
        torch.save(Xi.clone(),f'{data_dir}/test/{x_test_i}.pt')
        x_test_i += 1
torch.save(y_train_all,f'{data_dir}/y_train.pt')
torch.save(y_test_all,f'{data_dir}/y_test.pt')

100%|██████████| 32/32 [06:08<00:00, 11.50s/it]
